Importantly, there is a very, very, very tiny note in the problem statement that each time you end up with different possible choices of directions, they always end up at the same end point. This takes this problem from being exponential in size to being linear. Many of the functions recorded here enumerate my attempts to deal with the exponential version. Only the `make_graph` function is the necessary one in the end.

In [147]:
import itertools as its
from collections import defaultdict, deque

import networkx as nx

In [102]:
line = open('input.txt').read().strip()[1:-1]

In [157]:
class Choice:
    def __init__(self, toparse):
        assert toparse[0] == '('
        
        self.choices = []

        i = 1
        while i < len(toparse):
            
            new_tree = Tree(toparse[i:])
            i += new_tree.parsed_len
            self.choices.append(new_tree)
            assert i <= len(toparse)
            if i == len(toparse):
                break
                
            assert toparse[i] in [')', '|']
            
            if toparse[i] == '|':
                i += 1
                continue
            else:
                self.parsed_len = i + 1
                return

        self.parsed_len = i
        
    def max(self):
        return max(choice.max() for choice in self.choices) if self.choices else 0
    
    def descend(self, grid, dist=0, pos=(0, 0)):
        for choice in self.choices:
            for end_pos, end_dist in choice.descend(grid, dist=dist, pos=pos):
                yield end_pos, end_dist
                
    def make_graph(self, graph, pos):
        for choice in self.choices:
            new_pos = choice.make_graph(graph, pos)
        return new_pos
    
    def __bool__(self):
        return any(choice for choice in self.choices)
    
    def __repr__(self):
        return '|'.join(repr(choice) for choice in self.choices)
    
    def __iter__(self):
        for choice in self.choices:
            for c in choice:
                yield c

In [165]:
def step(pos, c):
    if c == 'N':
        return (pos[0], pos[1] - 1)
    if c == 'S':
        return (pos[0], pos[1] + 1)
    if c == 'W':
        return (pos[0] - 1, pos[1])
    if c == 'E':
        return (pos[0] + 1, pos[1])
    raise NotImplemented

class Tree:
    def __init__(self, toparse):
        self.begin = ''
        
        i = 0
        while i < len(toparse):
            c = toparse[i]
            if c == '(':
                self.middle = Choice(toparse[i:])
                
                i += self.middle.parsed_len
                self.end = Tree(toparse[i:])

                i += self.end.parsed_len
                self.parsed_len = i
                return
        
            if c in ['|', ')']:
                #from IPython.core.debugger import Tracer; Tracer()()
                self.parsed_len = i
                self.middle = None
                self.end = None
                return
            
            self.begin += toparse[i]
            i += 1
            
        if i == len(toparse):
            self.parsed_len = len(toparse)
            self.middle = None
            self.end = None
            
    def __repr__(self):
        if self.end:
            if self.middle:
                return '{}({}){}'.format(self.begin, repr(self.middle), repr(self.end))
            else:
                return '{}{}'.format(self.begin, self.end)
        else:
            if self.middle:
                return '{}({})'.format(self.begin, repr(self.middle))
            else:
                return '{}'.format(self.begin)


    def __iter__(self):
        val = self.begin
        if self.middle:
            if self.end:
                for middle, end in its.product(self.middle, self.end):
                    yield val + str(middle) + str(end)
            else:
                for middle in self.middle:
                    yield val + str(middle)
        else:
            if self.end:
                for end in self.end:
                    yield val + str(end)
            else:
                yield val

    def max(self):
        total = len(self.begin)
        total += self.middle.max() if self.middle else 0
        total += self.end.max() if self.end else 0
        return total
    
    def __bool__(self):
        return bool(self.begin) or bool(self.middle) or bool(self.end)
    
    def descend(self, grid, dist=0, pos=(0, 0)):
        grid[pos] = min(grid.get(pos, dist), dist)
        new_pos = pos
        for c in self.begin:
            new_pos = step(new_pos, c)
            dist += 1
            grid[new_pos] = min(grid.get(new_pos, dist), dist)

        # Now we've reached the end of our mandatory stepping
        if self.middle:
            if self.end:
                for mid_pos, mid_dist in self.middle.descend(grid, dist=dist, pos=new_pos):
                    for end_pos, end_dist in self.end.descend(grid, dist=mid_dist, pos=mid_pos):
                        yield end_pos, end_dist
            else:
                for mid_pos, mid_dist in self.middle.descend(grid, dist=dist, pos=new_pos):
                    yield mid_pos, mid_dist
        else:
            if self.end:
                for end_pos, end_dist in self.end.descend(grid, dist=dist, pos=new_pos):
                    yield end_pos, end_dist
            else:
                yield new_pos, dist
                
    def make_graph(self, graph, pos):
        graph.add_node(pos)
        for c in self.begin:
            new_pos = step(pos, c)
            graph.add_node(new_pos)
            graph.add_edge(pos, new_pos)
            pos = new_pos
            
        if self.middle:
            pos = self.middle.make_graph(graph, pos)
            
        if self.end:
            pos = self.end.make_graph(graph, pos)
            
        return pos

In [178]:
tree = Tree(line)
graph = nx.Graph()
tree.make_graph(graph, (0, 0))
shortest_paths = nx.shortest_path_length(graph, (0, 0))

print('The answer to part 1 is:', max(shortest_paths.values()))
print('The answer to part 2 is:', sum(v >= 1000 for v in shortest_paths.values()))

The answer to part 1 is: 3930
The answer to part 2 is: 8240
